# U-Net Model
In this week's assignment, you'll be using a network architecture called "U-Net". The name of this network architecture comes from it's U-like shape when shown in a diagram like this (image from [U-net entry on wikipedia](https://en.wikipedia.org/wiki/U-Net)):

<img src="images/U-net_example_wikipedia.png" alt="U-net Image" width="600"/>

U-nets are commonly used for image segmentation, which will be your task in the upcoming assignment. You won't actually need to implement U-Net in the assignment, but we wanted to give you an opportunity to gain some familiarity with this architecture here before you use it in the assignment.

As you can see from the diagram, this architecture features a series of down-convolutions connected by max-pooling operations, followed by a series of up-convolutions connected by upsampling and concatenation operations. Each of the down-convolutions is also connected directly to the concatenation operations in the upsampling portion of the network. For more detail on the U-Net architecture, have a look at the original [U-Net paper by Ronneberger et al. 2015](https://arxiv.org/abs/1505.04597).

In this lab, you'll create a basic U-Net using Keras. 

In [3]:
import keras
from keras import backend as K
# from keras.engine import Input, Model
from keras.layers import Conv3D, MaxPool3D, UpSampling3D, Activation, BatchNormalization, PReLU
from keras.optimizers import Adam

# set image shape to have the channles in the first dim
K.set_image_data_format('channels_first')

import tensorflow
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Conv3DTranspose
tensorflow.compat.v1.logging.set_verbosity(tensorflow.compat.v1.logging.ERROR)

## 1. The "Depth" of your U-Net
The "depth" of your U-Net is equal to the number of down-convolutions you will use. In the image above, the depth is 4 because there are 4 down-convolutions running down the left side including the very bottom of the U.

For this exercise, you'll use a U-Net depth of 2, meaning you'll have 2 down-convolutions in your network.

### 1.1 Input Layer and its "Depth"

In this lab and in the assignment, you will be doing 3D image segmentation, which is to say that, in addition to "height" and "width", your input layer will also have a "length". We are deliberately using the word "length" instead of "depth" here to describe the third spatial dimension of the input so as not to confuse it with the depth of the network as defined above.

The shape of the input layer is `(num_channels, height, width, length)`, where `num_channels` you can think of like color channels in an image, `height`, `width` and `length` are just the size of the input.

For the assignment, the values will be:
- num_channels: 4
- height: 160
- width: 160
- length: 16

In [4]:
# define input layer
input_layer = Input(shape=(4, 160, 160, 16))
input_layer

<KerasTensor: shape=(None, 4, 160, 160, 16) dtype=float32 (created by layer 'input_1')>

Notice that the tensor shape has a '?' as the very first dimension.  This will be the batch size. So the dimensions of the tensor are: (batch_size, num_channels, height, width, length)

## 2. Contracting (downward) Path
Here you'll start by constructing the downward path in your network (the left side of the U-Net).  The `(height, width, length)` of the input gets smaller as you move down this path, and the number of channels increases.

### 2.1 Depth 0

By "depth 0" here, we're referring to the depth of the first down-convolution in the U-net.

The number of filters is specified for each depth and for each layer within that depth.

The formula to use for calculating the number of filters is:
$$filters_{i} = 32 \times (2^{i})$$

Where $i$ is the current depth.

So at depth $i=0$:
$$filters_{0} = 32 \times (2^{0}) = 32$$

### 2.2 Layer 0
There are two convolutional layers for each depth

In [5]:
down_depth0_l0 = Conv3D(filters=32,
                        kernel_size=(3, 3, 3),
                        padding='same',
                        strides=(1, 1, 1))(input_layer)
down_depth0_l0

<KerasTensor: shape=(None, 32, 160, 160, 16) dtype=float32 (created by layer 'conv3d')>

Notice that with 32 filters, the result you get above is a tensor with 32 channels.

Run the next cell to add a relu activation to the first convolutional layer

In [6]:
relu0 = Activation('relu')(down_depth0_l0)
relu0

<KerasTensor: shape=(None, 32, 160, 160, 16) dtype=float32 (created by layer 'activation')>

### 2.3 Depth 0, Layer 1
For layer 1 of depth 0, the formula for calculating the number of filters is:
$$filters_{i} = 32 \times (2^{i}) \times 2$$

Where $i$ is the current depth.
- Notice that the '$\times~2$' at the end of this expression isn't there for layer 0.


So at depth $i=0$ for layer 1:
$$filters_{0} = 32 \times (2^{0}) \times 2 = 64$$


In [7]:
down_depth0_l1 = Conv3D(filters=64,
                        kernel_size=(3, 3, 3),
                        padding='same',
                        strides=(1, 1, 1))(down_depth0_l0)
down_depth0_l1 = Activation('relu')(down_depth0_l1)
down_depth0_l1

<KerasTensor: shape=(None, 64, 160, 160, 16) dtype=float32 (created by layer 'activation_1')>

### 2.4 Max pooling
Within the U-Net architecture, there is a max pooling operation after each of the down-convolutions (not including the last down-convolution at the bottom of the U). In general, this means you'll add max pooling after each down-convolution up to (but not including) the `depth - 1` down-convolution (since you started counting at 0).

For this lab exercise:
- The overall depth of the U-Net you're constructing is 2
- So the bottom of your U is at a depth index of: $2-1 = 1$.
- So far you've only defined the $depth=0$ down-convolutions, so the next thing to do is add max pooling

In [8]:
down_depth0_pool = MaxPool3D(pool_size=(2, 2, 2))(down_depth0_l1)
down_depth0_pool

<KerasTensor: shape=(None, 64, 80, 80, 8) dtype=float32 (created by layer 'max_pooling3d')>

#### 2.4.1 Depth 1, Layer 0

At depth 1, layer 0, the formula for calculating the number of filters is:
$$filters_{i} = 32 \times (2^{i})$$

Where $i$ is the current depth.

So at depth $i=1$:
$$filters_{1} = 32 \times (2^{1}) = 64$$

Run the next cell to add a Conv3D layer to your network with relu activation

In [9]:
down_depth1_l0 = Conv3D(filters=64,
                        kernel_size=(3, 3, 3),
                        padding='same',
                        strides=(1, 1, 1))(down_depth0_pool)
down_depth1_l0 = Activation('relu')(down_depth1_l0)
down_depth1_l0

<KerasTensor: shape=(None, 64, 80, 80, 8) dtype=float32 (created by layer 'activation_2')>

#### 2.4.2 Depth 1,  Layer 1

For layer 1 of depth 1 the formula you'll use for number of filters is:
$$filters_{i} = 32 \times (2^{i}) \times 2$$

Where $i$ is the current depth.
- Notice that the '$\times 2$' at the end of this expression isn't there for layer 0.

So at depth $i=1$:
$$filters_{0} = 32 \times (2^{1}) \times 2 = 128$$

Run the next cell to add another Conv3D with 128 filters to your network.

In [10]:
down_depth1_l1 = Conv3D(filters=128,
                        kernel_size=(3, 3, 3),
                        padding='same',
                        strides=(1, 1, 1))(down_depth1_l0)
down_depth1_l1 = Activation('relu')(down_depth1_l1)
down_depth1_l1

<KerasTensor: shape=(None, 128, 80, 80, 8) dtype=float32 (created by layer 'activation_3')>

#### No max pooling at depth 1 (the bottom of the U)

When you get to the "bottom" of the U-net, you don't need to apply max pooling after the convolutions.

## 3. Expanding  (upward) Path

Now you'll work on the expanding path of the U-Net, (going up on the right side, when viewing the diagram).  The image's (height, width, length) all get larger in the expanding path.

### 3.1 Depth 0, Up Sampling Layer 0

You'll use a pool size of (2,2,2) for upsampling.
- This is the default value for [tf.keras.layers.UpSampling3D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/UpSampling3D)
- As input to the upsampling at depth 1, you'll use the last layer of the downsampling.  In this case, it's the depth 1 layer 1.

Run the next cell to add an upsampling operation to your network.
Note that you're not adding any activation to this upsampling layer.